## Setup

In [0]:
!git clone https://github.com/javieraespinosa/big-data-analytics-practicals.git

In [0]:
%cd big-data-analytics-practicals/data 
%ls -d $PWD/*
%cd ../..

In [0]:
dbutils.fs.cp("file:///databricks/driver/big-data-analytics-practicals/data/appl_stock.csv", "dbfs:/appl_stock.csv")
dbutils.fs.cp("file:///databricks/driver/big-data-analytics-practicals/data/sales_info.csv", "dbfs:/sales_info.csv")

In [0]:
spark

# Introduction au Dataframe avec Spark


## 1 Présentation

Pour faciliter le développement de programme, Spark à partir de la version 2 intégre la notion de Dataframe.

Le DataFrame est une couche d'abstraction des RDD, qui présentent les données comme des tables de base de données sans se préoccuper de la taille des données.
Les DataFrames offrent de nombreux avantages:
* Une syntaxe beaucoup plus simple.
* Possibilité d'utiliser SQL directement dans la trame de données.
* La parallélisation des traitements dans une architecture distribuée est gérée par Spark.

Si vous avez utilisé R ou même la bibliothèque pandas avec Python, vous connaissez probablement déjà le concept des DataFrames. Même si Spark supporte plusieurs langages il faut savoir que le langage le moins efficace est le python.

Vous êtes prêt, jouons un peu avec les dataframes.

Let's get started!




## 1 - Les fichiers de données

Pour lire un fichier, il suffit d'utiliser la fonction associée à votre format de fichier:
* csv,txt -> load
* parquet -> parquet
* json -> json
...

In [0]:
df_csv = spark.read.load("dbfs:/sales_info.csv", format="csv", sep=",", inferSchema="true", header="true")

Dans les options de lecture du fichier, nous avons demandé à Spark d'utiliser le séparteur ","  et d'inférer le schéma.

Pour consulter le schéma évaluer par Spark utilisez la fonction printSchema()

In [0]:
df_csv.printSchema()

Pour voir les fonctions disponibles avec les dataframes consultez la documentation Spark :
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html

Pour avoir un aperçu des données, utilisez la méthode show()

In [0]:
df_csv.show()

Vous l'aurez compris Spark utilise un échantillonage des données pour évaluer le schema des données.

Si le schéma ne vous convient pas vous pouvez lui indiquer le schema que vous souhaitez (renommer les noms des colonnes par exemple).

In [0]:
from pyspark.sql.types import *
# Required for StructField, StringType, IntegerType, etc.


fields = [  StructField("Compagnie", StringType(), True),
            StructField("Personne", StringType(), True),
            StructField("Ventes", FloatType(), False)
            ]

csvSchema = StructType(fields)

In [0]:
df_csv_with_my_schema = spark.read.option("header", "true").option("delimiter", ",").schema(csvSchema).csv("dbfs:/sales_info.csv")

In [0]:
df_csv_with_my_schema.printSchema()

In [0]:
df_csv_with_my_schema.show()

Les fichiers csv et texte ne sont pas des formats adaptés pour de grosse volumétrie.
Une bonne pratique dans les environnements de BIG DATA est de travailler avec des formats binaires pour 2 raisons :
* il consomme moins de stockage.
* il est plus facile de déplacer des données via le réseau.

Dans les écosystèmes HADOOP, les données sont sérialisées sous 2 formes :
* Ligne : Avro, Kryo, Protobuff
* Column : Parquet, Orc

En entreprise, vous trouverez souvent les formats Avro ou parquet pour leur performance.

Pour convertir un fichier en parquet c'est aussi simple que ça :

In [0]:
df_csv_with_my_schema.write.format("parquet").save("dbfs:/sales_info.parquet")

In [0]:
dbutils.fs.ls("dbfs:/sales_info.parquet")

Vous pouvez remarquer qu'au format parquet, notre fichier sales_info.parquet est un répertoire contenant des fichiers part-00000, cela vous rappelle-t'il quelque chose ?
En effet, pour traiter des données parquet sait que les données vont être distribuées et par conséquent applique le principe de partitionnement. Pour un fichier de plusieurs Go, il est possible d'indiquer la taille des partitions parquet.

Bien entendu, la plupart des formats binaires inscrit le schéma des données dans chaque partition.

Vérifions cela :

In [0]:
Sales_parquet = spark.read.parquet("dbfs:/sales_info.parquet")

In [0]:
Sales_parquet.printSchema()

In [0]:
Sales_parquet.show()

On peut facilement convertir d'un format à un autre :

In [0]:
Sales_parquet.write.format('json').save('dbfs:/sales.json')

Pour lire ensuite vos données.

In [0]:
Sales_json = spark.read.json("dbfs:/sales.json")

In [0]:
Sales_json.show()

# 2 Exploiter les DataFrames

Pour manipuler les DataFrames, nous avons un ensemble de méthodes disponibles :

In [0]:
df = spark.read.parquet("dbfs:/sales_info.parquet")

In [0]:
dir(df)

Pour afficher les colonnes :

In [0]:
df.columns

Travaillons à présent sur les manipulations que nous pouvons faire :

In [0]:
dfColVentes = df.select('Ventes')

In [0]:
type(dfColVentes)

Le résultat retourné par le select est un DataFrame

In [0]:
df.select('Ventes').show()

Pour selectionner les N premières lignes, on trouvera la fonction head :

In [0]:
# Retourne une d'object RowReturns list of Row objects
df.head(2)

Pour sélectionner multiple colonne :

In [0]:
df.select(['Compagnie','Ventes']).show()

ou

In [0]:
df.select(df.Compagnie.alias('test'), df.Ventes).show()

Pour le renommage d'une colonne on peut utiliser la méthode alias :

In [0]:
df.select(df.Compagnie.alias('col1'), df.Ventes.alias('col2')).show()

Si on veut créer de nouvelles colonnes à partir d'une colonne existante  :

In [0]:
df.withColumn('VentesAdd', df['Ventes'] * 10).show()

Pour renommer une colonne

In [0]:
# Simple Rename
df.withColumnRenamed('Compagnie', 'Company').show()

On peut filtrer sur un critére :

In [0]:
df.filter(df.Personne=="Sam").show()

Pour les opérations d'agrégation, on utilisera les fonctions du module pyspark.sql.functions lequel regroupe les fonctions max, min, mean ...

In [0]:
import pyspark.sql.functions as f

df.groupBy('Compagnie').agg(f.max('Ventes').alias('Max Ventes'), f.min('Ventes').alias('Min Ventes'), f.mean('Ventes').alias('Moy_Ventes')).orderBy('Compagnie').show()

Comme pour les bases de données relationnelles, on peut faire des jointures entre des sources de données avec différents formats :

In [0]:
from pyspark.sql import *
# Création d'un dataframe des localisations des différentes compagnies.
# Le code ci-dessous indique comment créer un dataframe manuellement

Location = Row("Compagnie", "location")
location1 = Location('GOOG', 'US')
location2 = Location('MSFT', 'EUROPE')
location3 = Location('FB', 'ASIA')

locationRows = [location1, location2, location3]
df_location  = spark.createDataFrame(locationRows)
type(df_location)

On affiche le contenu de notre nouveau dataframe :

In [0]:
df_location.show()

On affiche la structure du dataframe :

In [0]:
df.printSchema()

On réalise la jointure entre le dataframe df listant toutes les ventes des companies et le dataframe df_location précisant la localisation des companies :

In [0]:
df.join(df_location, df_location.Compagnie == df.Compagnie, "left_outer").show()

### Utilisation du SQL

Pour utiliser des requêtes SQL directement sur un DataFrame, vous devrez l'enregistrer dans une vue temporaire:

In [0]:
# La méthode createOrReplaceTempView enregistre
# le DataFrame comme une view temporaire
df.createOrReplaceTempView("sales")

In [0]:
sql_sales = spark.sql("SELECT * FROM sales")

In [0]:
sql_sales

In [0]:
sql_sales.show()

Bien entendu, vous pouvez appliquer des requêtes SQL avec des conditions :

In [0]:
spark.sql("SELECT * FROM sales WHERE Ventes > 500").show()

Spark SQL supporte un sous ensemble de la norme SQL92.

Bravo !!!

Vous êtes prêt à faire vos premiers exercices avec Spark.

# Cleaning

In [0]:
dbutils.fs.rm("dbfs:/sales_info.parquet", recurse=True)
dbutils.fs.rm("dbfs:/sales.json", recurse=True)
dbutils.fs.rm("dbfs:/appl_stock.csv")
dbutils.fs.rm("dbfs:/sales_info.csv")